In [22]:
import pandas as pd
import bqplot
import numpy as np
import traitlets
import ipywidgets
import matplotlib.pyplot as plt
%matplotlib inline

Read in files

In [23]:
b = pd.read_csv("building_inventory.csv",
           na_values = {'Year Acquired': 0,
                       'Year Constructed': 0}) 

In [24]:
# b

Making a pivot table to summarize the data for 'Congress Dist' and 'Agency Name'

In [25]:
stats = pd.pivot_table(b,index = 'Congress Dist',columns='Agency Name',values = 'Square Footage',aggfunc =np.sum)
# sums = pd.pivot_table(b,index = ['Congress Dist'],values = ['Square Footage'],aggfunc =np.sum)
# sums.values
# type(stats.columns)
# stats.loc[0, 'Appellate Court / Fifth District']
# stats.index
stats

Agency Name,Appellate Court / Fifth District,Appellate Court / Fourth District,Appellate Court / Second District,Appellate Court / Third District,Chicago State University,Department of Agriculture,Department of Central Management Services,Department of Corrections,Department of Human Services,Department of Juvenile Justice,...,Illinois Emergency Management Agency,Illinois Medical District Commission,Illinois State University,Northeastern Illinois University,Northern Illinois University,Office of the Attorney General,Office of the Secretary of State,Southern Illinois University,University of Illinois,Western Illinois University
Congress Dist,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,231278.0,NaN,372784.0,NaN,...,5650.0,31200.0,NaN,NaN,41315.0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,1219492.0,NaN,NaN,NaN,449547.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49572.0,1253943.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3643049.0,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,9932.0,NaN,362890.0,NaN,...,NaN,NaN,NaN,1110103.0,NaN,NaN,28452.0,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72411.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,2088840.0,NaN,304039.0,NaN,...,NaN,15000.0,NaN,NaN,NaN,NaN,56904.0,NaN,6363904.0,NaN
8,NaN,NaN,43330.0,NaN,NaN,NaN,65268.0,NaN,913263.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# (I) CREATE LABEL
mySelectedLabel = ipywidgets.Label() # start with our label
# mySelectedLabel

In the head map, the data is calculated by log10, as the original data span is too large. Two axes are Congress Dist and Agency Name.

In [27]:
# (II) HEAT MAP
# (1) Scales: x/y, colors
col_sc = bqplot.ColorScale(scheme = "Reds")
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# (2) Axis: x/y, colors
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')
x_ax = bqplot.Axis(scale = x_sc)
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical')

# (3) Marks: heatmap
heat_map = bqplot.GridHeatMap(color = np.log10(stats.values),
                              row = stats.index,
                              column = stats.columns,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'}, # make interactive on click of each box
                              anchor_style = {'fill':'blue'}, # to make our selection blue
                              selected_style = {'opacity': 1.0}, # make 100% opaque if box is selected
                              unselected_style = {'opacity': 0.8}) # make a little see-through if not

Hard code the default data. [0,0] was put as default selection.

In [28]:
# (III) LINE PLOT
# (1) scales
x_scl = bqplot.LinearScale()
y_scl = bqplot.LinearScale()

# (2) Axis
ax_xcl = bqplot.Axis(label='Year Acquired', scale=x_scl)
ax_ycl = bqplot.Axis(label='Total Square Footage', scale=y_scl, 
                    orientation='vertical', side='left')

# (3) Marks
i, j = 0, 0
temp = b[(b['Congress Dist'] == i) & (b['Agency Name'] == stats.columns[j])]
g = temp.groupby(['Year Acquired'])['Square Footage'].sum()
# x = np.array(g.index)
# y = np.array(g)
footage_line = bqplot.Lines(x = np.array(g.index), y = np.array(g), scales={'x':x_scl, 'y':y_scl})

Define the on_select function. When selected, the line axis values are modified to the newly selected ones.

In [29]:
# (IV) LINKING TOGETHER DASHBOARD WITH INTERACTIVITY
def on_selected(change):
    if len(change['owner'].selected) == 1: 
        i, j = change['owner'].selected[0]
        v = stats.loc[i][j]
        mySelectedLabel.value = 'Data Value = ' + str(v)
        
        temp = b[(b['Congress Dist'] == i) & (b['Agency Name'] == stats.columns[j])]
        g = temp.groupby(['Year Acquired'])['Square Footage'].sum()
        footage_line.x = np.array(g.index)
        footage_line.y = np.array(g)
        
heat_map.observe(on_selected, 'selected')

Put all figures and lables together.

In [30]:
# (V) CREATE FIGURES
fig_heatmap = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax])
fig_sq = bqplot.Figure(marks = [footage_line], axes = [ax_xcl, ax_ycl])

fig_heatmap.layout.min_width='500px'
fig_sq.layout.min_width='500px'

In [31]:
myDashboard = ipywidgets.VBox([mySelectedLabel, ipywidgets.HBox([fig_heatmap,fig_sq])])
myDashboard

One of the data transformation I did was taking the log10 of the total square footage values, as the span of it is too large.A log based value can be more informative.

The NaN data was set to 0 when reading in the file.For the heat map and line plot, if there is no value to be shown, it will be displayed as black (in heat map) and empty (for line plot).

The colors were set to red and black for values and NaN. The size of the heat map and line plot was made to 500px so that they can be displayed side by side. Moreover, when selected, the opacity of the selected cell remains one, but that of other cells become 0.8 to show the difference. Last but not least, the line plot for emepty cells are also empty. However, for cells that only have one dot (only one of the acquired year has square footages), the line plot is also empty, as one dot does not form a line.